In [17]:
from torchstat import stat
from main_real import *
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from utils import k_fold
from thop import profile
from thop import clever_format

In [5]:
args = opts.parse_args(True)

In [26]:
args.noise = 0
args.dataset = 'MUTAG'
dataset_name, feat_str, _ = opts.create_n_filter_triples([args.dataset])[0]
dataset = get_dataset(dataset_name, sparse=True, feat_str=feat_str, root=args.data_root)
print(dataset)
train_idx, test_idx, val_idx = k_fold(dataset, args.folds, args.epoch_select)
test_dataset = dataset[test_idx[0].long()]
test_loader = DataLoader(test_dataset, args.batch_size, shuffle=False)

MUTAG(188)


In [10]:
test_loader = DataLoader(test_dataset, args.batch_size, shuffle=False)

Data(edge_index=[2, 38], x=[17, 109], y=[1], uny=[1, 2], split_n=[1], split_e=[1])

In [33]:
outs = []
for modelname in ["GCN", "GIN", "GAT","CausalGCN","CausalGIN", "CausalGAT"]:    
    args.model = modelname
    model_func = opts.get_model(args)
    model = model_func(dataset.num_features, dataset.num_classes).cpu()
    flops, params = profile(model, inputs=test_loader)
    flops, params = clever_format([flops, params], '%.3f')
    outs.append(f'{modelname}&{params}&{flops}')

[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_

In [28]:
flops, params

('1.963M', '36.319K')

In [34]:
for i in outs:
    print(i)

GCN&18.268K&1.051M
GIN&117.340K&36.736M
GAT&18.268K&1.051M
CausalGCN&36.319K&1.963M
CausalGIN&135.391K&37.647M
CausalGAT&36.319K&1.963M
